# Contract risk analysis

In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.14"
#r "nuget: Scriban"
#r "nuget: dotenv.net"

using Azure;
using Azure.AI.OpenAI;
using Scriban;

using dotenv.net;
DotEnv.Load();

var endpoint=Environment.GetEnvironmentVariable("OPENAI_URI");
var key=Environment.GetEnvironmentVariable("OPENAI_KEY");
var model=Environment.GetEnvironmentVariable("OPENAI_GPT_DEPLOYMENT");


Installed Packages Azure.AI.OpenAI, 1.0.0-beta.14 dotenv.net, 3.1.3 Scriban, 5.9.1

In [25]:
async Task<string> ProcessCompletion(string prompt, int max_tokens=500, float temperature=0.3f)
{
    OpenAIClient client = new(new Uri(endpoint), new AzureKeyCredential(key));

    

    var chatCompletionsOptions = new ChatCompletionsOptions()
    {
        DeploymentName = model, //This must match the custom deployment name you chose for your model
        Messages =
        {
            new ChatRequestAssistantMessage(prompt),
        },
        MaxTokens = max_tokens,
        Temperature = temperature
    };

    Response<ChatCompletions> response = await client.GetChatCompletionsAsync(chatCompletionsOptions);

    return response.Value.Choices[0].Message.Content;
}

In [9]:
string RenderTemplate(string template, dynamic data) {
    var templateObject = Template.Parse(template);
    return templateObject.Render(data);
}

In [10]:
List<Tuple<string,string>> songs = [
new Tuple<string,string>("La Canción de los Gatos",@"En el jardín, jugando van,
Pequeños gatos, con gran afán.
Pelos suaves, y orejas puntiagudas,
Saltan y juegan, ¡qué travesuras!

¡Miau, miau, los gatos juegan así!
En la luz del sol, o bajo la luna, sí.
¡Miau, miau, saltan con destreza,
Los gatos son la pura belleza!

Con bigotes finos, y ojos brillantes,
Exploran rincones, son tan elegantes.
Persiguen mariposas, atrapan ratones,
Los gatos son reyes de los callejones.

¡Miau, miau, los gatos juegan así!
En la luz del sol, o bajo la luna, sí.
¡Miau, miau, saltan con destreza,
Los gatos son la pura belleza!

Descansan en tejados, bajo el cielo estrellado,
Ronroneando suavemente, a veces hasta dormitando.
Cada gato, con su propia personalidad,
¡Son pequeños amigos llenos de vitalidad!

¡Miau, miau, los gatos juegan así!
En la luz del sol, o bajo la luna, sí.
¡Miau, miau, saltan con destreza,
Los gatos son la pura belleza!

Y así termina la canción de los gatos,
Con sus travesuras y sus saltos.
Gatitos felices, en su propio rincón,
¡Que la alegría de los gatos siga en tu corazón!"),
new Tuple<string,string>("Conquistando el Amor",@"En la penumbra de la noche, perdido en el laberinto,
Caminando entre susurros, buscando el amor instinto.
Ojos que brillan como estrellas, en la oscuridad se encuentran,
Corazones en batalla, donde las sombras se entrelazan.

Conquistando el amor, en un juego sin final,
Donde las promesas se tejen, como hilos en el cristal.
Entre suspiros y secretos, dos almas se entrelazan,
En este duelo de pasiones, el amor se abalanza.

En el campo de las emociones, donde la razón se desvanece,
Se libra la batalla, entre la dicha y la tristeza.
Susurros de seducción, en la danza de la pasión,
Labios que pronuncian versos, creando un lazo de unión.

Bajo el cielo estrellado, donde los sueños se conjugan,
Se forja la alianza, que en el corazón se anida.
Palabras como flechas, atraviesan el silencio,
Conquistando el amor, en un eterno encuentro.

Conquistando el amor, en un juego sin final,
Donde las promesas se tejen, como hilos en el cristal.
Entre suspiros y secretos, dos almas se entrelazan,
En este duelo de pasiones, el amor se abalanza.

En el jardín de los sentimientos, donde florece la esperanza,
Se escribe la historia, de una conquista que avanza.
Manos que se buscan, en la penumbra del deseo,
Conquistando el amor, como un fuego que no teme.

Bajo el manto de la Luna, sellando el pacto eterno,
Dos corazones en victoria, en este amor moderno.
Conquistando el amor, como héroes en la trama,
En este cuento sin final, donde el amor se proclama."),
];

### Prepare the prompt template

In [29]:
var promptTemplate = @"system:
You are an assistant can help determine how easy it would be for an English speaker to learn to sing a song in Spanish. Easy songs have straightforward vocabulary and grammar and avoid complex sentence structures, metaphors, poetic structures and language, and uncommon words. Songs with familiar or universal themes, such as love, emotions, or everyday activities, can be easier for learners to relate to and understand. It helps when the context of the song is relatable to the listener. 
Rate the following song lyrics in Spanish from 1-10, with 10 being the hardest for an English speaker to learn.

user:
Song Title: ===
{{title}}
===

Lyrics: ===
{{input}}
===

Output format:
{ ""score"":-1, ""title"":"""",""explanation"": """"}

Provide an explanation in one sentence. Output in JSON format only.";

### Get the scores for each song

In [30]:
foreach(var song in songs) 
{
    var prompt = RenderTemplate(promptTemplate, new { title = song.Item1, input = song.Item2 });
    //Console.WriteLine(prompt);
    Console.WriteLine(await ProcessCompletion(prompt));
}

{ "score":3, "title":"La Canción de los Gatos", "explanation": "The song features simple vocabulary and grammar, with a relatable and universal theme of cats playing, which makes it relatively easy for an English speaker to learn." }
{ "score":8, "title":"Conquistando el Amor", "explanation": "The song uses metaphorical language, poetic structures, and some less common vocabulary, making it more challenging for an English speaker to learn." }
